In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from prophet.plot import plot_plotly, plot_components_plotly
from prophet import Prophet
from datetime import datetime
from time import time

In [135]:
#read in csv for
all_bdrm = pd.read_csv("housing_prices_data.csv")
all_bdrm.dropna(inplace=True)
all_bdrm.head(54)

,RegionID,Neighborhood,State,City,Metro,County,2019-01-31,2019-02-28,2019-03-31,2019-04-30,...,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,BedroomCount,Location
0,118208,South Los Angeles,CA,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,331301.0,328698.0,325906.0,323279.0,...,486580.0,486336.0,486796.0,486585.0,487578.0,486809.0,489862.0,489152.0,1,"South Los Angeles, Los Angeles"
1,268496,Southeast Los Angeles,CA,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,300152.0,297244.0,293939.0,290796.0,...,413743.0,416449.0,419431.0,420848.0,422551.0,422515.0,427628.0,430679.0,1,"Southeast Los Angeles, Los Angeles"
2,273565,East San Jose,CA,San Jose,"San Jose-Sunnyvale-Santa Clara, CA",Santa Clara County,418566.0,415034.0,410266.0,406200.0,...,465629.0,463605.0,460273.0,456309.0,452782.0,449427.0,451437.0,454010.0,1,"East San Jose, San Jose"
3,273088,Bullard,CA,Fresno,"Fresno, CA",Fresno County,93733.0,94532.0,94801.0,94473.0,...,156623.0,158448.0,159079.0,159166.0,158239.0,157523.0,162523.0,167592.0,1,"Bullard, Fresno"
4,276652,West San Jose,CA,San Jose,"San Jose-Sunnyvale-Santa Clara, CA",Santa Clara County,538711.0,530457.0,519145.0,508979.0,...,550898.0,546854.0,542083.0,538433.0,536506.0,535151.0,551396.0,573673.0,1,"West San Jose, San Jose"
5,276215,Blossom Valley,CA,San Jose,"San Jose-Sunnyvale-Santa Clara, CA",Santa Clara County,462816.0,458452.0,451249.0,445176.0,...,506927.0,504596.0,500951.0,496958.0,494183.0,492298.0,494323.0,495488.0,1,"Blossom Valley, San Jose"
6,276476,North Long Beach,CA,Long Beach,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,266728.0,265556.0,263175.0,260926.0,...,402911.0,404267.0,405088.0,405569.0,406738.0,406488.0,410371.0,411345.0,1,"North Long Beach, Long Beach"
7,118987,Willow Glen,CA,San Jose,"San Jose-Sunnyvale-Santa Clara, CA",Santa Clara County,581363.0,573486.0,563685.0,553849.0,...,644741.0,637962.0,630263.0,624112.0,621834.0,620207.0,631158.0,639936.0,1,"Willow Glen, San Jose"
8,273480,Downtown,CA,San Jose,"San Jose-Sunnyvale-Santa Clara, CA",Santa Clara County,648736.0,643617.0,635750.0,626482.0,...,677957.0,671321.0,663768.0,656509.0,651720.0,648991.0,651081.0,653072.0,1,"Downtown, San Jose"
9,113886,Boyle Heights,CA,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,376977.0,372352.0,365782.0,360569.0,...,466802.0,467986.0,469767.0,470172.0,470898.0,470543.0,476385.0,478590.0,1,"Boyle Heights, Los Angeles"


In [136]:
#Choose your city
city = "San Diego"
#Choose room size
bedroom_count = 2


In [137]:
#if unsure choose a Max budget to see options
max_budget = 800000
min_budget = 600000

In [138]:
def budget_options (max_budget,min_budget,city):
    options_df = all_bdrm[all_bdrm['2023-02-28'] >= max_budget]
    options_df = all_bdrm[all_bdrm['2023-02-28'] <= min_budget]
    options_df = all_bdrm.loc[all_bdrm['City']==city]
    options = options_df['Neighborhood']
    
    return options

In [139]:
budget_options(max_budget,min_budget,city)

39              University City
42              Rancho Bernardo
43                Carmel Valley
51                Pacific Beach
54                   North Park
                 ...           
5373    Roseville - Fleet Ridge
5445                  Otay Mesa
5506                   La Playa
5513              Sunset Cliffs
5578           Rancho Encantado
Name: Neighborhood, Length: 368, dtype: object

In [140]:
#create a variable for the city to look into 
neighborhood = "La Jolla"

In [141]:
#chose the first row in as a test 
grouped_df = all_bdrm.loc[all_bdrm['City']==city]
grouped_df = grouped_df.loc[grouped_df['BedroomCount']==bedroom_count]
grouped_df

,RegionID,Neighborhood,State,City,Metro,County,2019-01-31,2019-02-28,2019-03-31,2019-04-30,...,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,BedroomCount,Location
346,116625,Mira Mesa,CA,San Diego,"San Diego-Chula Vista-Carlsbad, CA",San Diego County,424434.0,432344.0,442574.0,450659.0,...,600551.0,598425.0,593020.0,587616.0,582384.0,577728.0,575514.0,572852.0,2,"Mira Mesa, San Diego"
374,118673,University City,CA,San Diego,"San Diego-Chula Vista-Carlsbad, CA",San Diego County,564041.0,568504.0,574193.0,578121.0,...,758313.0,756330.0,749817.0,743031.0,737899.0,732900.0,728095.0,722561.0,2,"University City, San Diego"
377,117557,Rancho Penasquitos,CA,San Diego,"San Diego-Chula Vista-Carlsbad, CA",San Diego County,385066.0,391040.0,397487.0,402046.0,...,568730.0,566776.0,561417.0,555430.0,550039.0,545639.0,543534.0,542638.0,2,"Rancho Penasquitos, San Diego"
379,275127,Rancho Bernardo,CA,San Diego,"San Diego-Chula Vista-Carlsbad, CA",San Diego County,474327.0,478181.0,484261.0,488982.0,...,686021.0,682669.0,676552.0,670379.0,665061.0,659328.0,655611.0,651964.0,2,"Rancho Bernardo, San Diego"
380,273140,Carmel Valley,CA,San Diego,"San Diego-Chula Vista-Carlsbad, CA",San Diego County,597616.0,605570.0,615596.0,622997.0,...,876179.0,871714.0,862881.0,853807.0,847104.0,841484.0,838485.0,836306.0,2,"Carmel Valley, San Diego"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411,403239,Fox Canyon,CA,San Diego,"San Diego-Chula Vista-Carlsbad, CA",San Diego County,357773.0,364474.0,373778.0,381018.0,...,545862.0,543606.0,538432.0,533358.0,530054.0,527121.0,525625.0,524526.0,2,"Fox Canyon, San Diego"
1416,268203,Harborview,CA,San Diego,"San Diego-Chula Vista-Carlsbad, CA",San Diego County,786467.0,789147.0,795313.0,798897.0,...,1091839.0,1086400.0,1070911.0,1057569.0,1047768.0,1038540.0,1027563.0,1020311.0,2,"Harborview, San Diego"
1548,268215,Horton Plaza,CA,San Diego,"San Diego-Chula Vista-Carlsbad, CA",San Diego County,1436169.0,1428991.0,1419751.0,1412633.0,...,1740340.0,1739649.0,1734285.0,1730069.0,1729313.0,1720811.0,1720278.0,1713369.0,2,"Horton Plaza, San Diego"
1581,274893,Old Town,CA,San Diego,"San Diego-Chula Vista-Carlsbad, CA",San Diego County,532071.0,550188.0,575799.0,595280.0,...,681863.0,677442.0,669095.0,662409.0,658504.0,655957.0,661638.0,665957.0,2,"Old Town, San Diego"


In [142]:
#transposed data 
grouped_df = grouped_df.transpose()
#reset the index while moving the current index into a new column
grouped_df.reset_index(inplace=True)

In [143]:
#change column name to the neighborhood row 1
neighborhoods = grouped_df.iloc[1]
#rename the column
grouped_df.rename(columns=neighborhoods, inplace=True)
grouped_df

,Neighborhood,Mira Mesa,University City,Rancho Penasquitos,Rancho Bernardo,Carmel Valley,Pacific Beach,North Park,La Jolla,Otay Mesa West,...,Mission Valley West,Mission Beach,Little Italy,Sunset Cliffs,Cortez Hill,Fox Canyon,Harborview,Horton Plaza,Old Town,Gaslamp Quarter
0,RegionID,116625,118673,117557,275127,273140,117156,274717,46087,403253,...,268312,116638,268259,118387,273355,403239,268203,268215,274893,273808
1,Neighborhood,Mira Mesa,University City,Rancho Penasquitos,Rancho Bernardo,Carmel Valley,Pacific Beach,North Park,La Jolla,Otay Mesa West,...,Mission Valley West,Mission Beach,Little Italy,Sunset Cliffs,Cortez Hill,Fox Canyon,Harborview,Horton Plaza,Old Town,Gaslamp Quarter
2,State,CA,CA,CA,CA,CA,CA,CA,CA,CA,...,CA,CA,CA,CA,CA,CA,CA,CA,CA,CA
3,City,San Diego,San Diego,San Diego,San Diego,San Diego,San Diego,San Diego,San Diego,San Diego,...,San Diego,San Diego,San Diego,San Diego,San Diego,San Diego,San Diego,San Diego,San Diego,San Diego
4,Metro,"San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA",...,"San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA","San Diego-Chula Vista-Carlsbad, CA"
5,County,San Diego County,San Diego County,San Diego County,San Diego County,San Diego County,San Diego County,San Diego County,San Diego County,San Diego County,...,San Diego County,San Diego County,San Diego County,San Diego County,San Diego County,San Diego County,San Diego County,San Diego County,San Diego County,San Diego County
6,2019-01-31,424434.0,564041.0,385066.0,474327.0,597616.0,611447.0,526152.0,646929.0,309073.0,...,441539.0,1011937.0,591175.0,1131558.0,592588.0,357773.0,786467.0,1436169.0,532071.0,576109.0
7,2019-02-28,432344.0,568504.0,391040.0,478181.0,605570.0,618919.0,529571.0,661128.0,315673.0,...,443372.0,1031533.0,592071.0,1160884.0,592160.0,364474.0,789147.0,1428991.0,550188.0,573270.0
8,2019-03-31,442574.0,574193.0,397487.0,484261.0,615596.0,630861.0,535105.0,679750.0,324267.0,...,446360.0,1063199.0,594075.0,1202197.0,591736.0,373778.0,795313.0,1419751.0,575799.0,569120.0
9,2019-04-30,450659.0,578121.0,402046.0,488982.0,622997.0,640788.0,539096.0,694877.0,331981.0,...,447412.0,1093101.0,594917.0,1243602.0,591454.0,381018.0,798897.0,1412633.0,595280.0,565377.0


In [144]:
#dropping string columns 
grouped_df = grouped_df.drop(grouped_df.index[0:6])
#dropping bedroom count and reset columns
grouped_df = grouped_df.drop(grouped_df.index[50:52])
grouped_df = grouped_df.reset_index(drop=True)
grouped_df.tail()

,Neighborhood,Mira Mesa,University City,Rancho Penasquitos,Rancho Bernardo,Carmel Valley,Pacific Beach,North Park,La Jolla,Otay Mesa West,...,Mission Valley West,Mission Beach,Little Italy,Sunset Cliffs,Cortez Hill,Fox Canyon,Harborview,Horton Plaza,Old Town,Gaslamp Quarter
45,2022-10-31,587616.0,743031.0,555430.0,670379.0,853807.0,905405.0,718009.0,1060104.0,423712.0,...,618939.0,1403869.0,770790.0,1681387.0,716990.0,533358.0,1057569.0,1730069.0,662409.0,710213.0
46,2022-11-30,582384.0,737899.0,550039.0,665061.0,847104.0,899280.0,712015.0,1053546.0,422571.0,...,617663.0,1396641.0,764061.0,1675952.0,713579.0,530054.0,1047768.0,1729313.0,658504.0,707802.0
47,2022-12-31,577728.0,732900.0,545639.0,659328.0,841484.0,894110.0,706189.0,1048121.0,421490.0,...,615922.0,1393595.0,757765.0,1670109.0,709493.0,527121.0,1038540.0,1720811.0,655957.0,704659.0
48,2023-01-31,575514.0,728095.0,543534.0,655611.0,838485.0,895426.0,703837.0,1054309.0,423291.0,...,612386.0,1404061.0,752897.0,1678218.0,705635.0,525625.0,1027563.0,1720278.0,661638.0,700293.0
49,2023-02-28,572852.0,722561.0,542638.0,651964.0,836306.0,899077.0,701317.0,1063081.0,420899.0,...,608915.0,1411730.0,750368.0,1684986.0,703035.0,524526.0,1020311.0,1713369.0,665957.0,697780.0


In [145]:
grouped_df.rename(columns = {"Neighborhood":"ds"}, inplace=True)

In [147]:
#define the function to prep the data 
def one_city(dataframe, neighborhood):
    # create a new dataframe with the ds column and the specified index column
    new_df = pd.DataFrame({
        'ds': dataframe['ds'],
        #rename the column to y to fit in model
        "y": dataframe[neighborhood]
        
    })
    return new_df

In [148]:
city_df = one_city(grouped_df, neighborhood)

In [149]:
#split model to test and train
city_df = one_city(grouped_df, neighborhood)
train_model = city_df.iloc[:len(city_df)-6]
test_model = city_df.iloc[len(city_df)-6:]

In [150]:
model =Prophet()
model.fit(train_model)
future = model.make_future_dataframe(periods=6, freq='MS') #MS for monthly, H for hourly
forecast = model.predict(future)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/rayzapien/opt/anaconda3/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.




Initial log joint probability = -54.7666
Iteration  1. Log joint probability =    54.3889. Improved by 109.155.
Iteration  2. Log joint probability =     81.152. Improved by 26.7631.
Iteration  3. Log joint probability =    98.6366. Improved by 17.4847.
Iteration  4. Log joint probability =    100.493. Improved by 1.85622.
Iteration  5. Log joint probability =    102.585. Improved by 2.09184.
Iteration  6. Log joint probability =    102.638. Improved by 0.0528797.
Iteration  7. Log joint probability =    102.705. Improved by 0.0670639.
Iteration  8. Log joint probability =    102.719. Improved by 0.0140773.
Iteration  9. Log joint probability =    102.792. Improved by 0.0731136.
Iteration 10. Log joint probability =    102.877. Improved by 0.0848148.
Iteration 11. Log joint probability =    102.898. Improved by 0.0210929.
Iteration 12. Log joint probability =    102.955. Improved by 0.0577193.
Iteration 13. Log joint probability =    103.116. Improved by 0.16059.
Iteration 14. Log joi

/Users/rayzapien/opt/anaconda3/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/rayzapien/opt/anaconda3/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [151]:
plot_plotly(model, forecast)

In [167]:
#create a comparable dataframe to find r-square score
compared_df = forecast[["ds","trend","yhat"]].set_index("ds")

compared_df.tail(10)

,trend,yhat
ds,,
2022-05-31,1.053853e+06,1.064875e+06
2022-06-30,1.075832e+06,1.081511e+06
2022-07-31,1.098544e+06,1.093251e+06
2022-08-31,1.121256e+06,1.102568e+06
2022-09-01,1.121988e+06,1.119265e+06
2022-10-01,1.143968e+06,1.136875e+06
2022-11-01,1.166680e+06,1.154740e+06
2022-12-01,1.188659e+06,1.179772e+06
2023-01-01,1.211371e+06,1.207722e+06


In [168]:
city_df.tail(10)

,ds,y
40,2022-05-31,1073718.0
41,2022-06-30,1089356.0
42,2022-07-31,1093443.0
43,2022-08-31,1086183.0
44,2022-09-30,1072355.0
45,2022-10-31,1060104.0
46,2022-11-30,1053546.0
47,2022-12-31,1048121.0
48,2023-01-31,1054309.0
49,2023-02-28,1063081.0


In [153]:
compared_df.dropna(inplace=True)

In [154]:
#use r2_score from sklearn
from sklearn.metrics import mean_squared_error, r2_score
r2_score(compared_df.trend,compared_df.yhat)

0.9976536233294411